# TradingClient example

## Prepare client

In [1]:
# run this if you want to run the example from the examples folder without installing the package
import sys
sys.path.insert(0, "../")

In [2]:
# import the package
from osmosispy.trading_client import BinanceTradingClient, ITradingClient, TradeData
import time

don't forget to set your API key and secret to the environment variables
`BINANCE_API_KEY` and `BINANCE_API_SECRET` respectively.

```bash
export BINANCE_API_KEY=qwerty
export BINANCE_API_SECRET=asdfgh
```

In [3]:
# create a binance trading client
binance_client: ITradingClient = BinanceTradingClient.from_env()

Alternatively, you can pass them to the client constructor.

In [ ]:
# create a client from the constructor
binance_client: ITradingClient = BinanceTradingClient(
    api_key="qwerty",
    api_secret="asdfgh",
)

## Get osmosis trading info

In [ ]:
fee = binance_client.get_trading_fee("OSMOUSDT")
# fee object will contain the following fields:
# maker_commision: float
# taker_commision: float

print(f"Trading fee: {fee}")

In [ ]:
price_in_usdt = binance_client.get_price(symbol = "OSMOUSDT")

print(f"Current price in USDT:\n{price_in_usdt}")

## Listen trades

In [ ]:
# create trade callback
def trade_callback(trade: TradeData):
    print("===== OSMO USDT trade =====")
    print(f"price: {trade.price.amount}\nquantity: {trade.quantity.amount}")

# start listening to trades
binance_client.listen_trades(cb=trade_callback, symbol="OSMOUSDT")

In [ ]:
# listen for 60 seconds
time.sleep(60)

In [ ]:
# stop listening to trades
binance_client.stop_listeners()